# Pipeline Bronze → Silver com Delta Lake

Este notebook demonstra, em alto nível, um pipeline de transformação de dados da camada **Bronze** para a camada **Silver** em um Data Lake, utilizando **Apache Spark** e **Delta Lake**.


In [ ]:
# Importa as classes e funções necessárias do PySpark que serão utilizados no pipeline.

from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    LongType, StringType, TimestampType, DoubleType
)
from pyspark.sql.functions import (
    col, to_date, hour,
    split, array_distinct,
    regexp_extract,
    year, month
)

In [ ]:
# Inicializa a SparkSession já configurada para trabalhar com Delta Lake.
# Nesta etapa, o ambiente de execução distribuída é criado, permitindo que
# o pipeline leia, transforme e escreva dados de forma paralela.

spark = (SparkSession.builder
         .appName("BronzeToSilver-Delta-Full")
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .config("spark.sql.shuffle.partitions", "128")
         .getOrCreate())

In [ ]:
# Define a função de utilidade 'get_paths', responsável por centralizar os caminhos
# de leitura (camada Bronze) e escrita (camada Silver) no Data Lake.
def get_paths():
    storage = "tccprojectdlstorage"
    
    # O '*.parquet' indica que todos os arquivos Parquet na camada Bronze
    # serão lidos em um único DataFrame, independentemente de subpastas lógicas.
    bronze = f"abfss://bronze@{storage}.dfs.core.windows.net/*.parquet"
    
    # O destino é uma tabela única na camada Silver, representada como uma tabela Delta.
    silver = f"abfss://silver@{storage}.dfs.core.windows.net/iot_events_delta"
    return bronze, silver

In [ ]:
# Define o schema esperado para os dados da camada Bronze, garantindo consistência tipada
# entre execuções, evita inferência automática (que pode ser custosa) e facilita
# o controle de evolução de campos ao longo do tempo.

schema = StructType([
    StructField("event_id", LongType()),
    StructField("device_id", StringType()),
    StructField("device_type", StringType()),
    StructField("location_id", StringType()),
    StructField("event_ts", TimestampType()),
    StructField("temperature", DoubleType()),
    StructField("pressure", DoubleType()),
    StructField("energy_consumption", DoubleType()),
    StructField("battery_level", DoubleType()),
    StructField("status_code", StringType()),
    StructField("tags", StringType()),
    StructField("payload", StringType())
])

In [ ]:
# Implementa a função 'load_bronze', responsável por ler todos os dados Parquet
# da camada Bronze usando o schema definido. A opção 'mergeSchema' permite que o
# Spark una pequenas variações de schema entre arquivos, o que é útil em cenários
# onde o modelo de dados evolui com o tempo.

def load_bronze(path):
    print(f"Lendo TODOS os dados Bronze de: {path}")
    return (spark.read
            .schema(schema)
            .option("mergeSchema", "true")
            .parquet(path))

In [ ]:
# Define a função de transformação principal, responsável por converter os dados
# da camada Bronze em um modelo Silver mais analítico e limpo.
#
# Nesta etapa:
# - Cria colunas derivadas de tempo (data, ano, mês, hora);
# - Aplica regras de qualidade de dados (Data Quality) em temperatura, pressão e bateria;
# - Normaliza o campo de tags em um array de strings únicas;
# - Extrai metadados estruturados a partir do campo 'payload' usando expressões regulares.

def transform(df):
    print("Iniciando transformações (Silver)...")
    
    # Cria colunas derivadas de tempo para facilitar filtros, agregações e particionamento.
    df = df.withColumn("event_date", to_date(col("event_ts")))
    df = df.withColumn("event_year", year(col("event_ts")))
    df = df.withColumn("event_month", month(col("event_ts")))
    df = df.withColumn("event_hour", hour(col("event_ts")))
    
    # Aplica regras de qualidade para identificar leituras inválidas ou suspeitas.
    df = df.withColumn("dq_temp_out_of_range", (col("temperature") < -50) | (col("temperature") > 100))
    df = df.withColumn("dq_pressure_out_of_range", col("pressure") <= 0)
    df = df.withColumn("dq_battery_out_of_range", (col("battery_level") < 0) | (col("battery_level") > 100))
    
    # Converte a string de tags em um array de tags únicas, removendo duplicatas.
    df = df.withColumn("tags_array", array_distinct(split(col("tags"), ",")))
    
    # Extrai metadados estruturados a partir do campo 'payload' com expressões regulares.
    df = df.withColumn("fw_version", regexp_extract(col("payload"), r"fw=([^;]+);", 1))
    df = df.withColumn("meta_location_group", regexp_extract(col("payload"), r"location_group=([^;]+);", 1))
    df = df.withColumn("meta_device_group", regexp_extract(col("payload"), r"device_group=([^;]+);", 1))
    
    print("Transformações concluídas.")
    return df

In [ ]:
# Implementa a função 'write_silver', que materializa os dados transformados
# em uma tabela Delta na camada Silver. Nesta escrita:
# - Sobrescrever o conjunto de dados existente (modo 'overwrite');
# - Particionar fisicamente por ano e mês do evento (event_year, event_month),
#   melhorando a eficiência de consultas filtradas por tempo;
# - Utilizar o formato Delta, que adiciona controle transacional e suporte a ACID;
# - Permitir a evolução de schema com a opção 'overwriteSchema'.

def write_silver(df, path):
    print(f"Iniciando escrita para a camada Silver (Delta Lake) em: {path}")
    (df.write
     .mode("overwrite")
     .partitionBy("event_year", "event_month")
     .format("delta")
     .option("overwriteSchema", "true")
     .save(path))
    print("Escrita na camada Silver concluída.")

In [ ]:
# Executa o pipeline fim-a-fim, encadeando as funções de leitura, transformação
# e escrita. Esta célula representa o fluxo operacional:
# 1. Descobrir caminhos das camadas Bronze e Silver;
# 2. Ler todos os dados da Bronze em um DataFrame;
# 3. Aplicar transformações e regras de qualidade para gerar a Silver;
# 4. Persistir o resultado em uma tabela Delta particionada por ano e mês.

bronze_path, silver_path = get_paths()

df_bronze = load_bronze(bronze_path)
df_silver = transform(df_bronze)
write_silver(df_silver, silver_path)

print("Pipeline B->S (Delta) finalizado com sucesso.")